In [501]:
import tabula #Extracting tables from pdf
#from pdfminer.high_level import extract_pages, ectract_text
import os
import re
from tika import parser # pip install tika

import pandas as pd
import numpy as np

In [502]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [507]:
cd Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet/

/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet


In [508]:
path = os.listdir("/Users/Moni/Ironhack/my_ironhack/projects/final-project/data/historico-facturas-pdf/palacet")

# Extract name of documents
files = [i for i in path if ".DS" not in i]

#for testing
#files = files[3:5]

In [509]:
files

['6598904.pdf',
 '8994847.pdf',
 '12549682.pdf',
 '10076002.pdf',
 '11572189.pdf',
 '8135057.pdf',
 '16395407.pdf',
 '7394922.pdf',
 '6947878.pdf']

In [510]:
df = pd.DataFrame(columns=['description', 'units', 'pvp', "date", "order_num"])

for pdf in files:
    
    order_num = pdf.replace(".pdf", "")
    
    #parser the text in the pdf (returns a dict)
    raw = parser.from_file(pdf)
    
    # Extract the content (returns a str)
    content = raw['content']
    
    # He know there is only one date in the document (and the format)
    pattern = r'\b\d{2}/\d{2}/\d{2}\b'  #meets the date format
    
    # Extract the date - returns a list of one elemnt
    date = re.findall(pattern, content)[0]
    
    #Extract tables from the pdf:
    tables = tabula.read_pdf(pdf, pages="all", pandas_options={'header': None})
    
    #We know last table is always resume of ticket.exclude:
    #tables = tables[:-1]
    
    #Information about products(main table) in different pages, are extracted as different tables:
    for table in tables:
        
        if table[0][0] == "IVA":
            continue
        
        table.rename(columns={0: "description", 1:"units", 2:"pvp"}, inplace=True)
        table["date"] = date
        table["order_num"] = order_num

        result = pd.concat([df, table])

        df = result
print("pdf tables extrated")

pdf tables extrated


In [511]:
df

,description,units,pvp,date,order_num
0,Friegasuelos pino Bosque Verde líquido,4,"2,72 €",16/09/20,6598904
1,Papel multiusos resistente Bosque Verde,1,"2,89 €",16/09/20,6598904
2,Papel higiénico doble rollo Bosque Verde,3,"11,46 €",16/09/20,6598904
3,Lavavajillas aloe Bosque Verde concentrado,4,"6,20 €",16/09/20,6598904
4,Suavizante ropa talco Bosque Verde concentrado,2,"3,58 €",16/09/20,6598904
...,...,...,...,...,...
13,Activador del lavado ropa blanca Bosque Verde ...,1,"1,60 €",07/11/20,6947878
14,Suavizante ropa talco Bosque Verde concentrado,2,"3,58 €",07/11/20,6947878
15,Lavavajillas aloe Bosque Verde concentrado,1,"1,55 €",07/11/20,6947878
16,Lejía con detergente Bosque Verde,1,"0,71 €",07/11/20,6947878


In [512]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 17
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  134 non-null    object
 1   units        129 non-null    object
 2   pvp          129 non-null    object
 3   date         134 non-null    object
 4   order_num    134 non-null    object
dtypes: object(5)
memory usage: 6.3+ KB


In [485]:
def drop_na(df, column):
    nans = df[df[column].isna()].shape[0]
    df.dropna(subset=['description'], inplace = True)
    
    print(f"{nans} rows droped in colum {column}")
    
    return

In [486]:
drop_na(df, "description")

13 rows droped in colum description


In [487]:
# Vamos a ver que datos podemos encontrar en la columna 3:
not_nan_3 = df[df[3].isna() == False]

not_nan_3.sample()

,description,units,pvp,date,order_num,3
28,Papel Multiusos Resistente Bosque Verde,NaN,1.0,13/07/22,996926,"3,45 €"


In [488]:
#Eliminamos estas filas (luego las vamos a volver a incluir)
df = df[df[3].isna() == True]
df.shape

(779, 6)

In [489]:
# Vemos que es como si se hubieram desplazado, lo reognanizamos: 
not_nan_correct = not_nan_3
not_nan_correct["units"] = not_nan_3["pvp"]
not_nan_correct["pvp"] = not_nan_3[3]
not_nan_correct[3] = np.nan

/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_9684/2995014200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["units"] = not_nan_3["pvp"]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_9684/2995014200.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_nan_correct["pvp"] = not_nan_3[3]
/var/folders/k4/q8g97p495cx7f63kbs0wzyc00000gn/T/ipykernel_9684/2995014200.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [490]:
not_nan_correct.sample()
df = pd.concat([df, not_nan_correct])

In [491]:
df.shape # Se mantien el numpero de columnas por lo que lo hemos hecho bien

(827, 6)

In [493]:
df = df.drop(3, axis=1)

In [494]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 827 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  827 non-null    object
 1   units        803 non-null    object
 2   pvp          803 non-null    object
 3   date         827 non-null    object
 4   order_num    827 non-null    object
dtypes: object(5)
memory usage: 38.8+ KB


In [496]:
df.dropna(subset=['units'], inplace=True)

In [497]:
df

,description,units,pvp,date,order_num
0,Jamón serrano Incarlopsa lonchas,1,"3,49 €",01/10/20,302316
1,Queso fresco de cabra Hacendado,1,"2,20 €",01/10/20,302316
2,Queso rallado 4 quesos Hacendado,1,"1,59 €",01/10/20,302316
3,Perlas de mozzarella fresca Hacendado,1,"1,70 €",01/10/20,302316
4,Mozzarella di bufala campana Hacendado,1,"1,70 €",01/10/20,302316
...,...,...,...,...,...
14,Contramuslos de pollo deshuesados y sin piel,1.0,"4,28 €",19/10/21,337881
15,Solomillos de pollo,1.0,"5,19 €",19/10/21,337881
16,Filetes de dorada,1.0,"6,77 €",19/10/21,337881
17,Aguacates,1.0,"2,13 €",19/10/21,337881


In [498]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 803 entries, 0 to 18
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  803 non-null    object
 1   units        803 non-null    object
 2   pvp          803 non-null    object
 3   date         803 non-null    object
 4   order_num    803 non-null    object
dtypes: object(5)
memory usage: 37.6+ KB


In [499]:
export_to_csv(df,"facturas_family")

DataFrame successfully exported to facturas_family.csv
